In [11]:
from enum import Enum
import random

## Simple Example
> player has a 50% chance of hitting a HR or getting an OUT.

In [12]:
def play_inning():
    hrs = 0
    number_of_outs = 0

    while number_of_outs < 3:
        if random.randint(0, 1) == 1:
            hrs += 1
        else:
            number_of_outs += 1

    return hrs

outcomes = 0
iterations = 1000
for _ in range(1000):
    outcomes += play_inning()

outcomes / iterations ## expected === 3.0

2.958

## Runs Created Example

In [13]:
class EventCodes(Enum):
    Strikeout = 1
    Walk = 2
    HBP = 3
    Error = 4
    LongSingle = 5
    MediumSingle = 6
    ShortSingle = 7
    ShortDouble = 8
    LongDouble = 9
    Triple = 10
    HR = 11
    GIDP = 12
    NormalGroundBall = 13
    LineDriveInfieldFly = 14
    LongFly = 15
    MediumFly = 16
    ShortFly = 17

Single = [EventCodes.Walk, EventCodes.HBP, EventCodes.LongSingle, EventCodes.MediumSingle, EventCodes.ShortSingle]
Double = [EventCodes.ShortDouble, EventCodes.LongDouble]
OnBase = Single + [EventCodes.Error]
SingleOut = [
    EventCodes.Strikeout,
    EventCodes.ShortFly,
    EventCodes.MediumFly,
    EventCodes.LongFly,
    EventCodes.LineDriveInfieldFly,
    EventCodes.NormalGroundBall,
    EventCodes.GIDP
]

def execute_event_on_state(state: dict, event: EventCodes) -> dict:
    if event == event.GIDP:
        if state['bases'] == [1, 0, 0] or state['bases'] == [1, 1, 0] or state['bases'] == [1, 0, 1] or state['bases'] == [1, 1, 1]:
            state['outs'] += 1
        else:
            ## not a double play scenario
            event = EventCodes.NormalGroundBall

    if event in SingleOut:
        state['outs'] += 1

    if state['outs'] == 3:
        return state

    if event == EventCodes.Error:
        state['bases'] = [1] + state['bases']

    elif event == EventCodes.NormalGroundBall:
        if state['bases'] == [0,0,1]:
            state['bases'] = [0] + state['bases']

        if state['bases'] == [0,1,0]:
            state['bases'] = [0] + state['bases']

    elif event == EventCodes.GIDP:
        if state['bases'] == [1, 0, 0]:
            state['bases'] = [0, 0, 0]

        elif state['bases'] == [1, 1, 0]:
            state['bases'] = [1, 0, 0]

        elif state['bases'] == [1, 0, 1]:
            state['bases'] = [0, 0] + state['bases'][1:]

        elif state['bases'] == [1, 1, 1]:
            state['bases'] = [0, 1, 1]

    elif event in Single:
        if event == EventCodes.LongSingle:
            state['bases'] = [0] + state['bases']

        if event == EventCodes.MediumSingle:
            state['bases'] = state['bases'][:1] + [0, 0] + state['bases'][1:]
    
        state['bases'] = [1] + state['bases']

    elif event == EventCodes.LongFly:
        if state['bases'][2] == 1:
            ## third advances
            state['bases'] = state['bases'][:2] + [0] + state['bases'][2:]

        elif state['bases'][1] == 1:
            ## second advances
            state['bases'] = state['bases'][:1] + [0] + state['bases'][1:]

    elif event == EventCodes.MediumFly:
        state['bases'] = state['bases'][:2] + [0] + state['bases'][2:]

    elif event in Double:
        if event == EventCodes.LongDouble:
            state['bases'] = [0] + state['bases']

        state['bases'] = [0, 1] + state['bases']

    elif event == EventCodes.Triple:
        state['bases'] = [0, 0, 1] + state['bases']

    elif event == EventCodes.HR:
        state['bases'] = [0, 0, 0, 1] + state['bases']

    state['runs'] += sum(state['bases'][3:])
    state['bases'] = state['bases'][:3]

    return state


execute_event_on_state({ 'bases': [1,1,1], 'runs': 0, 'outs': 0}, EventCodes.GIDP)

{'bases': [0, 1, 1], 'runs': 0, 'outs': 2}

In [14]:
def create_historical_record(current_state, event):
    outcome = current_state.copy()

    outcome['event'] = event
    outcome['desc'] = f'{event.name}'

    return outcome

In [15]:
def play_half_inning():
    current_state = {
        'bases': [0, 0, 0],
        'runs': 0,
        'outs': 0,
    }

    history = []
    while current_state['outs'] < 3:

        event = random.choice(list(EventCodes.__members__.values()))
        current_state = execute_event_on_state(current_state, event)

        history.append(
            create_historical_record(current_state, event)
        )

    return history

play_half_inning()

[{'bases': [0, 0, 0],
  'runs': 0,
  'outs': 1,
  'event': <EventCodes.LineDriveInfieldFly: 14>,
  'desc': 'LineDriveInfieldFly'},
 {'bases': [0, 0, 0],
  'runs': 0,
  'outs': 2,
  'event': <EventCodes.MediumFly: 16>,
  'desc': 'MediumFly'},
 {'bases': [1, 0, 0],
  'runs': 0,
  'outs': 2,
  'event': <EventCodes.LongSingle: 5>,
  'desc': 'LongSingle'},
 {'bases': [1, 1, 0],
  'runs': 0,
  'outs': 2,
  'event': <EventCodes.ShortSingle: 7>,
  'desc': 'ShortSingle'},
 {'bases': [1, 1, 1],
  'runs': 0,
  'outs': 2,
  'event': <EventCodes.ShortSingle: 7>,
  'desc': 'ShortSingle'},
 {'bases': [1, 1, 1],
  'runs': 0,
  'outs': 3,
  'event': <EventCodes.LongFly: 15>,
  'desc': 'LongFly'}]